In [3]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp

from scipy.optimize import differential_evolution
from scipy.stats.qmc import Halton

# General imports
import numpy as np

from qiskit.quantum_info import SparsePauliOp

# custom module
from susy_qm import calculate_Hamiltonian

In [4]:
potential = 'QHO'
cutoff = 2

In [5]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)

eigenvalues, eigenvectors = np.linalg.eig(H)
min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = np.asarray(eigenvectors[:, min_index])

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [21]:
def ansatz(params, wire):
        qml.RY(params, wires=wire)

In [18]:
print(qml.draw(ansatz)(0.0,0))

0: ─╭|Ψ⟩──RY(0.00)─┤  
1: ─╰|Ψ⟩───────────┤  


In [22]:
def test(params1, params2):
    qml.BasisState([0]*2*num_qubits, wires=range(2*num_qubits))
    ansatz(params1, wire=[0])
    ansatz(params2, wire=[num_qubits])

In [23]:
print(qml.draw(test)(0.0,0.0))

0: ─╭|Ψ⟩──RY(0.00)─┤  
1: ─├|Ψ⟩───────────┤  
2: ─├|Ψ⟩──RY(0.00)─┤  
3: ─╰|Ψ⟩───────────┤  


In [26]:
shots=2
dev = qml.device('default.qubit', wires=2*num_qubits, shots=shots)
@qml.qnode(dev)
def destructive_swap_test(params1, params2):

    qml.BasisState([0]*2*num_qubits, wires=range(2*num_qubits))
    
    ansatz(params1, wire=[0])
    ansatz(params2, wire=[num_qubits])

    qml.Barrier()

    for i in range(num_qubits):
        qml.CNOT(wires=[i, num_qubits + i])
        qml.Hadamard(wires=i)

    probs = qml.probs(wires=range(2*num_qubits))

    return probs


In [27]:
print(qml.draw(destructive_swap_test)(0.0,0.0))

0: ─╭|Ψ⟩──RY(0.00)──||─╭●──H────┤ ╭Probs
1: ─├|Ψ⟩────────────||─│──╭●──H─┤ ├Probs
2: ─├|Ψ⟩──RY(0.00)──||─╰X─│─────┤ ├Probs
3: ─╰|Ψ⟩────────────||────╰X────┤ ╰Probs


In [29]:
params1=[3.1422955718061014]
params2=[0.1422955718061014]
destructive_swap_test(params1,params2)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])